# Educational Performance Data Preprocessing
**Author:** [elia.matsumoto@fgv.br] (2026)

## Overview
This notebook processes educational performance data for São Paulo municipality from 2000-2021, preparing it for temporal trend analysis and visualization.

## Input File
- `Data_2b_EDU_original.xlsx` containing three sheets:
  - `ABAND`: Student abandonment/dropout rates
  - `APROV`: Student approval/pass rates
  - `REPROV`: Student repetition/failure rates

## Output Files
1. `Data_2b_EDU_preproc.xlsx`: Cleaned dataset in long format with columns:
   - SUBPREFECTURE
   - YEAR (2000-2021)
   - ABAND, APROV, REPROV values
2. `Fig_2b_EDU_Data.jpeg`: Visualization of mean annual trends

## Processing Pipeline
1. **Data Loading**: Read three educational metrics from Excel
2. **Reshaping**: Convert from wide (years as columns) to long format (years as rows)
3. **Aggregation**: Calculate annual means across all subprefectures
4. **Transformation**: Apply log scale to normalize metric distributions
5. **Visualization**: Plot trends with highlighted interruption period (2013-2015)

## Key Variables
- **ABAND**: Student abandonment/dropout rates
- **APROV**: Student approval/pass rates
- **REPROV**: Student repetition/failure rates

## Time Series
- **Period**: 2000-2021 (22 years)
- **Interruption**: Gray shaded area marks 2013-2015 data gap

## Notes
- Data represents subprefecture-level aggregates
- Log transformation handles zero values (replaces -∞ with NaN)
- Visualization shows annual means across all subprefectures

In [ ]:
# Importing libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [ ]:
# Files info
Org_EDU_File   = 'Data_2b_EDU_original.xlsx';
ABAND_Sheet    = 'ABAND';
APROV_Sheet    = 'APROV';
REPROV_Sheet   = 'REPROV';
Data_EDU_File  = 'Data_2b_EDU_preproc.xlsx';
Data_EDU_Sheet = 'Data_preproc';

In [ ]:
# Reading Sao Paulo municipality EDU info
rawdata   = pd.ExcelFile(Org_EDU_File);
df_ABAND  = rawdata.parse(ABAND_Sheet);
df_APROV  = rawdata.parse(APROV_Sheet);
df_REPROV = rawdata.parse(REPROV_Sheet);

In [ ]:
# Initialization
years         = range(2000, 2022);
nyears        = len(years);
subpref       = df_ABAND['SUBPREFECTURE'].to_numpy().reshape(-1,1)
nSP           = len(subpref);
SUBPREFECTURE = np.tile(subpref, (nyears,1));
nTotal        = len(SUBPREFECTURE);
YEAR          = np.repeat(years, nSP).reshape(-1, 1) 

In [ ]:
# EDU numbers
FIELDS = range(1,nyears+1)
ABAND  = np.vstack([df_ABAND.iloc[:, idx].values for idx in FIELDS])
ABAND  = ABAND.reshape(-1, 1)
APROV  = np.vstack([df_APROV.iloc[:, idx].values for idx in FIELDS])
APROV  = APROV.reshape(-1, 1)
REPROV = np.vstack([df_REPROV.iloc[:, idx].values for idx in FIELDS])
REPROV = REPROV.reshape(-1, 1)

In [ ]:
# Output Data File
df_out = pd.DataFrame({
    'SUBPREFECTURE': SUBPREFECTURE.flatten(),
    'YEAR' : YEAR.flatten(),
    'ABAND': ABAND.flatten(),
    'APROV': APROV.flatten(),
    'REPROV': REPROV.flatten()    
})
df_out.to_excel(Data_EDU_File, sheet_name=Data_EDU_Sheet, index=False)

In [ ]:
# Grouping: mean by year
df = pd.DataFrame({
    'YEAR' : YEAR.flatten(),
    'ABAND': ABAND.flatten(),
    'APROV': APROV.flatten(),
    'REPROV': REPROV.flatten()    
})
df_mean = df.groupby('YEAR').mean()[['ABAND', 'APROV', 'REPROV']].reset_index()

In [ ]:
# Log scale
df_mean['ABAND']  = np.log(df_mean['ABAND'])
df_mean['ABAND']  = df_mean['ABAND'].replace(-np.inf, np.nan)
df_mean['APROV']  = np.log(df_mean['APROV'])
df_mean['APROV']  = df_mean['APROV'].replace(-np.inf, np.nan)
df_mean['REPROV'] = np.log(df_mean['REPROV'])
df_mean['REPROV'] = df_mean['REPROV'].replace(-np.inf, np.nan)

In [ ]:
# Plotting mean curves
plt.figure(figsize=(10, 6))
# Plot each curve
plt.plot(df_mean['YEAR'], df_mean['ABAND'], label='ABAND Mean (log scale)', marker='o', linewidth=1)
plt.plot(df_mean['YEAR'], df_mean['APROV'], label='APROV Mean (log scale)', marker='s', linewidth=1)
plt.plot(df_mean['YEAR'], df_mean['REPROV'], label='REPROV Mean (log scale)', marker='^', linewidth=1)
plt.axvspan(2013, 2015, alpha=0.1, color='gray',label='Interruption')
# Add labels and title
plt.xlabel('YEAR', fontsize=12)
plt.xticks(years, years, rotation=45)
plt.ylabel('Mean Value (log scale)', fontsize=12)
plt.title('Mean Values of ABAND, APROV, and REPROV by Year', fontsize=14, fontweight='bold')
# Add legend
plt.legend(fontsize=11)
# Add grid for better readability
plt.grid(True, alpha=0.3)
# Adjust layout and display
plt.tight_layout()
# Save plot
Fig_File = 'Fig_2b_EDU_Data.jpeg';
plt.savefig(Fig_File, format='jpeg', dpi=300)
plt.show()